In [6]:
!pip install cmake
!pip install dlib
!pip install face_recognition

import os

사람의 사진을 단 한 장만 넣어주면, 동영상 내에서 그 사람의 얼굴만 검출하는 것을 만들어봤습니다. 저는 제 사진을 주고, 영상 내에서 제 얼굴만 검출되도록 예제를 만들어 봤습니다. face_recognition 라이브러리와 opencv-python 라이브러리를 활용했습니다. 또한 face_recognition 라이브러리를 설치하기 전에, dlib을 설치하였는데 gpu세팅을 따로 해주지 않았습니다.<br><br>

특정 사람의 얼굴만 검출하는 프로그램의 알고리즘은 아래와 같습니다.
 - 입력해 준 사진 속에서 얼굴을 검출합니다.<br>
 - 그 얼굴에서 특성 벡터를 도출합니다.<br><br>
 
 - 웹캠으로 촬영되는 프레임 내 얼굴을 모두 검출합니다.<br>
 - 검출된 얼굴에서 특성 벡터를 도출합니다.<br><br>
 
 - 검출된 얼굴의 특성 벡터와 입력해 준 사진 속 얼굴의 특성 벡터가 서로 비슷한지 거리로 판단합니다.<br>
 - 비슷하다면(도출한 특성 간 거리가 가까우면), 그 사람의 얼굴이라고 판단하고 검출된 얼굴 주변에 빨간 박스를 그려줍니다 <br>

In [ ]:
import cv2
import face_recognition
 
# 사전 학습할 이미지를 불러옵니다.
image_to_be_matched = face_recognition.load_image_file('aa.jpeg')
name = "HaSeungJun"
 
# 로드된 벡터를 특징 벡터로 인코딩합니다.
image_to_be_matched_encoded = face_recognition.face_encodings(image_to_be_matched)[0]
print(image_to_be_matched_encoded)
 
# 웹캠을 실행합니다.
webcam = cv2.VideoCapture(0)
 
# 웹캠 실행한다면 찾을수없다는 메세지와 꺼줍니다.
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
 
# loop through frames
while webcam.isOpened():
 
    # 웹캠으로 부터 프레임을 읽어 옵니다.
    status, frame = webcam.read()
 
    # 불러오지 못한다며 해당 경고 메세지를 출력합니다.
    if not status:
        print("Could not read frame")
        exit()
 
    # face_locations = face_recognition.face_locations(frame) # HoG 기반 얼굴 검출기
    # HOG란 History Of Gradient 의 약자 이미지 경계의 기울기 벡터 크기(magnitude)와 방향(direction)을 히스토그램으로 나타내 계산
    face_locations = face_recognition.face_locations(frame, number_of_times_to_upsample=0, model="cnn") # CNN 기반 얼굴 검출기
 
    for face_location in face_locations:
 
        # 해당하는 이미지의 각 면의 위치를 프린트합니다.
        top, right, bottom, left = face_location
 
        # 다음과 같이 실제 자신의 얼굴에 엑세스합니다.
        face_image = frame[top:bottom, left:right]
 
        try:
            face_encoded = face_recognition.face_encodings(face_image)[0]
            result = face_recognition.compare_faces([image_to_be_matched_encoded], face_encoded, 0.5)
 
            if result[0] == True:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                Y = top - 10 if top - 10 > 10 else top + 10
                text = name
                cv2.putText(frame, text, (left, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        except:
            pass
 
    # 화면의 출력문입니다.
    cv2.imshow("detect me", frame)
 
    # pQ를 누르면 종료됩니다.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# release resources
webcam.release()
cv2.destroyAllWindows()


[-1.33859843e-01  9.84298438e-02  5.55847287e-02 -2.36537177e-02
 -4.57476601e-02 -2.80992985e-02 -1.84654370e-02 -9.27140415e-02
  1.18476138e-01 -1.26641169e-01  1.94185495e-01 -8.99155289e-02
 -2.15813875e-01 -1.01910703e-01 -1.80970579e-02  1.80116594e-01
 -1.95827067e-01 -1.07511714e-01 -5.33360206e-02  1.10064223e-02
  1.43980503e-01 -2.81653591e-02 -2.04345919e-02  3.02258357e-02
 -5.51578403e-02 -3.20436418e-01 -7.89341927e-02 -6.12371117e-02
  3.37810777e-02 -2.03463137e-02 -8.22747126e-03  2.12935358e-03
 -1.95113286e-01 -5.10891080e-02 -4.00288701e-02  2.18387693e-04
 -1.56712309e-02 -7.30503649e-02  1.88645259e-01 -5.11382744e-02
 -2.19681889e-01 -2.00953782e-02  4.33793478e-02  1.94216147e-01
  1.22042149e-01  6.15659840e-02  2.03926321e-02 -1.00240216e-01
  1.10904545e-01 -1.97533995e-01 -1.12246070e-03  1.27963841e-01
  1.00082308e-01  7.60797337e-02  7.58442283e-03 -1.60053566e-01
  2.30107009e-02  5.66400811e-02 -7.05291107e-02  2.28480473e-02
  5.69931976e-02 -7.38192